# `Open Sooq` Web Scraping

In [41]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [42]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0"}

In [43]:
base_url = "https://om.opensooq.com/en/property/property-for-rent"
current_page_url = rent_url

In [44]:
property_data = {
                "propert_title": [], 
                "propert_location": [], 
                "price": [], 
                "area": [], 
                "listing_type": []
}

In [48]:
page_number = 1
while True:
    print(f"scarpping page {page_number}...")
    try:
        response = requests.get(current_page_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"error fetching page {current_page_url}: {e}")
        break

    soup = BeautifulSoup(response.content, "html.parser")
    properties = soup.find_all('div', class_="postDet flex flex-1 flexWrap flexSpaceBetween flexDirectionColumn overflowHidden ripple")
    if not properties:
        print("No Property Found!")
        break
    for card in properties:
        title = card.find("div", class_="flex flexSpaceBetween gap-10")
        property_data["propert_title"].append(title.text.strip() if title else None)

        location = card.find("div", class_="flex alignItems gap-5 darkGrayColor")
        property_data["propert_location"].append(location.text.strip() if location else None)   

        price = card.find("div", class_="priceColor bold alignSelfCenter font-18 ms-auto")
        property_data["price"].append(price.text.strip() if price else None)
        
        size = None
        p_tag = card.find("p")
        if p_tag and p_tag.text:
            match = re.search(r"Surface\s*Area:\s*([\d,]+)\s*m2", p_tag.text, re.IGNORECASE)
            if match:
                size = match.group(1).replace(",", "")
        property_data["area"].append(size)

        property_data["listing_type"].append("For Rent")
        
    next_button = soup.find('a', class_='blueHoverBg border pagerItem flex alignItems justifyContent radius-8 ripple', attrs={'title':"Go to next page"}, href=True)
    if next_button:
        next_page_relative_url = next_button["href"]
        new_url = requests.compat.urljoin(base_url, next_page_relative_url)

    if new_url == current_page_url:
        print("Reached the last page. No more properties found!")
        break
    else:
        current_page_url = new_url
        page_number += 1
        time.sleep(1)

scarpping page 1...
scarpping page 2...
scarpping page 3...
scarpping page 4...
scarpping page 5...
scarpping page 6...
scarpping page 7...
scarpping page 8...
scarpping page 9...
scarpping page 10...
scarpping page 11...
scarpping page 12...
scarpping page 13...
scarpping page 14...
scarpping page 15...
scarpping page 16...
scarpping page 17...
scarpping page 18...
scarpping page 19...
scarpping page 20...
scarpping page 21...
scarpping page 22...
scarpping page 23...
scarpping page 24...
scarpping page 25...
scarpping page 26...
scarpping page 27...
scarpping page 28...
scarpping page 29...
scarpping page 30...
scarpping page 31...
scarpping page 32...
scarpping page 33...
scarpping page 34...
scarpping page 35...
scarpping page 36...
scarpping page 37...
scarpping page 38...
scarpping page 39...
scarpping page 40...
scarpping page 41...
scarpping page 42...
scarpping page 43...
scarpping page 44...
scarpping page 45...
scarpping page 46...
scarpping page 47...
scarpping page 48...
s

In [49]:
df = pd.DataFrame(property_data)
df

,propert_title,propert_location,price,area,listing_type
0,120000 m2 3 Bedrooms Apartments for Rent in Dh...,"Dhofar, Salala",35 OMR,120000,For Rent
1,3 Bedrooms Chalet for Rent in Dhofar Salala,"Dhofar, Salala",,250,For Rent
2,10 m2 Studio Apartments for Rent in Muscat Amerat,"Muscat, Amerat",7 OMR,10,For Rent
3,Furnished Daily in Muscat Al Mawaleh,"Muscat, Al Mawaleh",8 OMR,None,For Rent
4,للإيجار مكتب تجاري واجهة على شارع مزون - معرض...,"Muscat, Al Khoud",600 OMR,125,For Rent
...,...,...,...,...,...
5455,For Rent: Office Spaces from 70–240 SQM Prime...,"Muscat, Hamriya",275 OMR,70,For Rent
5456,قوس ورد للمدخل للإيجار shop entrance decor,"Muscat, Ruwi",50 OMR,None,For Rent
5457,جهاز حمام مغربي,"Muscat, Ruwi",120 OMR,None,For Rent
5458,300 m2 4 Bedrooms Townhouse for Rent in Muscat...,"Muscat, Qurm",450 OMR,300,For Rent


In [50]:
df.to_csv("OpenSooq.csv", index=False)